<a href="https://colab.research.google.com/github/CarsonHhs/InventoryManagement/blob/main/OilPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

In [115]:
df = pd.read_csv('/content/drive/MyDrive/Colab/Price.csv',parse_dates= ['Date'],dayfirst=True)
#df.set_index('Date', inplace=True)
#df =df.resample('m').sum()
#df.reset_index(inplace=True)

In [116]:
#df['Date'] = pd.to_datetime(df['Date'])
#df = df.reindex(index= df.index[::-1])

In [117]:
#zaman = np.arange(1, len(df) + 1, 1)
OHCL_avg = df.mean(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  


In [118]:
OHCL_avg = np.reshape(OHCL_avg.values, (len(OHCL_avg),1)) 

In [119]:
scaler = MinMaxScaler(feature_range=(0,1))
OHCL_avg = scaler.fit_transform(OHCL_avg)

In [120]:
#print(OHCL_avg)
train_OHLC = int(len(OHCL_avg)*0.9)
test_OHLC =  int(len(OHCL_avg)*0.1)

train_OHLC, test_OHLC = OHCL_avg[0:train_OHLC,:], OHCL_avg[train_OHLC:len(OHCL_avg),:]

In [121]:
#output = activation(dot(input, kernel) + bias)

def new_dataset(dataset, step_size):
    data_X, data_Y = [], []
    for i in range(len(dataset)-step_size-1):
        a = dataset[i:(i+step_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + step_size, 0])
    return np.array(data_X), np.array(data_Y)

In [273]:
#Train the datasets and test it
trainX, trainY = new_dataset(train_OHLC,4)
testX, testY = new_dataset(test_OHLC,4)

In [274]:
trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1,testX.shape[1]))

step_size = 4

In [278]:
testX

(38, 1, 4)

In [232]:
model = Sequential()
model.add(LSTM(128, input_shape=(1, step_size)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('linear'))

In [233]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=25, verbose=2)

Epoch 1/10
16/16 - 2s - loss: 0.0842
Epoch 2/10
16/16 - 0s - loss: 0.0264
Epoch 3/10
16/16 - 0s - loss: 0.0125
Epoch 4/10
16/16 - 0s - loss: 0.0094
Epoch 5/10
16/16 - 0s - loss: 0.0071
Epoch 6/10
16/16 - 0s - loss: 0.0056
Epoch 7/10
16/16 - 0s - loss: 0.0047
Epoch 8/10
16/16 - 0s - loss: 0.0045
Epoch 9/10
16/16 - 0s - loss: 0.0043
Epoch 10/10
16/16 - 0s - loss: 0.0041


In [234]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])

testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])


In [272]:
trainPredict[0]

array([17.996288], dtype=float32)

In [269]:
a = np.empty((step_size,))
a [:]= np.nan
b = np.empty((int(testY.size+6),))
b [:]= np.nan
c=trainPredict[0]
d = np.concatenate((a, c))
d= np.concatenate((d,b))
df['Train'] = d

ValueError: ignored

In [ ]:
!pip install "notebook>=5.3" "ipywidgets>=7.5"

In [268]:
df.head(10)

,Date,Price,Train
0,1986-01-01,22.93,NaN
1,1986-01-02,15.46,NaN
2,1986-01-03,12.61,NaN
3,1986-01-04,12.84,NaN
4,1986-01-05,15.38,15.38
5,1986-01-06,13.43,13.43
6,1986-01-07,11.59,11.59
7,1986-01-08,15.10,15.10
8,1986-01-09,14.87,14.87
9,1986-01-10,14.90,14.90


In [ ]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df.Date), y=list(df.High)))

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)



In [ ]:
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()